<a href="https://colab.research.google.com/github/yg-li/QA-KG-RL/blob/master/QA_KG_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --no-cache-dir torch-scatter torch-sparse torch-cluster
!pip install torch-geometric

!pip uninstall -y spacy
!pip install spacy==2.1.3
# !spacy download en_core_web_lg
!pip install neuralcoref

     |████████████████████████████████| 716kB 4.6MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-io3nfr0c/wheels/da/50/84/0bae0cf7e1bcf3cc97f8ebea2e5ef8e95d021ef3eb002fdd3e
  Stored in directory: /tmp/pip-ephem-wheel-cache-io3nfr0c/wheels/9d/83/0a/38ea460df5586a075b877fe089619e5238487712a0645940bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-io3nfr0c/wheels/ee/54/4a/eb079ec61377a93ea07aba9eea53adcae41450111b54de0118
Successfully built torch-scatter torch-sparse torch-cluster
     |████████████████████████████████| 102kB 4.8MB/s 
     |████████████████████████████████| 348kB 11.0MB/s 
     |████████████████████████████████| 51kB 28.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/cc/52/03/4ffa09bfe63693035a218fd50b059a9ad0f1638d5d9a7a86f2
  Stored in directory: /root/.cache/pip/wheels/91/3e/ee/e5630ef0fd53cedaa6e911ba27e8b40fff034388d1f264bb92
Successfully built torch-geometric plyfile
Uninstalling spacy-2.1.4:
  Successfully uninstalled spacy-2.1.4
     |█████████████

In [0]:
import torch
from torch_geometric.data import Data

import spacy
nlp = spacy.load("en_core_web_sm")
# import en_core_web_lg
# nlp = en_core_web_lg.load()

import neuralcoref
neuralcoref.add_to_pipe(nlp)

import networkx as nx

import os
import json
import itertools
import re

In [0]:
# mount Google Drive
from google.colab import drive
drive.mount('/gdrive')
data_path = '/gdrive/My Drive/Colab Notebooks/CSML/Project/data/qangaroo_v1.1/wikihop'

# read in QAngaroo WikiHop
with open(os.path.join(data_path, 'dev.json')) as f:
  src = json.loads(f.read())

In [0]:
instance = 0

# build graphs
query = src[instance].get('query')
docs = [nlp(text.lower()) for text in src[instance].get('supports')]
cands = src[instance].get('candidates')
cands = dict(zip(cands, range(len(cands))))
answer = src[instance].get('answer')
# print([(w.text, w.pos_) for w in docs[0]])
# docs[0].ents

In [0]:
# print(docs[1])
# print([w.pos_ for w in docs[1]])
# print(' '.join([w.lemma_ for w in docs[1]]))
# print("Noun phrases:", [chunk.text for chunk in docs[1].noun_chunks])
# print("PROPN:", [token.lemma_ for token in docs[1] if token.pos_ == "PROPN"])

In [0]:
def build_entity_graph(cands, docs):
  ''' build the entity graph used in Entity-GCN ''' 
  nodes = {}
  with_edges = set()
  
  doc_based_edges = set()
  match_edges = set()
  coref_edges = set()
  compl_edges = set()
  
  # accumulate nodes, add the doc_based & coreference edges
  for doc in docs:
    coref_clusters = [[m.text for m in c.mentions] for c in doc._.coref_clusters]
    text = doc.text
    doc_tmp = set()
    coref_tmps = [set()] * len(coref_clusters)
    for cand in cands:
      matches = re.findall(r'\b%s\b' % cand, text)
      new_nodes = range(len(nodes), len(nodes)+len(matches))
      doc_tmp.update(new_nodes)
      nodes.update(dict(zip(new_nodes, [cand]*len(matches))))
      
      for cluster, i in zip(coref_clusters, range(len(coref_clusters))):
        if cand in cluster:
          coref_tmps[i].update(new_nodes)
          
    for pair in itertools.combinations(doc_tmp, 2):
      doc_based_edges.add(pair)
      with_edges.update(pair)
    for coref_tmp in coref_tmps:
      for pair in itertools.combinations(coref_tmp, 2):
        coref_edges.add(pair)
        with_edges.update(pair)
      
  # add exact match edges
  for i, j in itertools.combinations(nodes, 2):
    if nodes[i] == nodes[j]:
      match_edges.add((i,j))
      with_edges.update((i,j))
      
  # add complement edges
  isolated_nodes = set(nodes) - with_edges
  if isolated_nodes:
    for pair in itertools.combinations(isolated_nodes, 2):
      compl_edges.add(pair)
      
  return nodes, doc_based_edges, match_edges, coref_edges, compl_edges

In [0]:
nodes, doc_based_edges, match_edges, coref_edges, compl_edges = build_entity_graph(cands, docs)